# Mixed Integer Programming(MIP) for the MnM Project

In [2]:
import glob, os
import numpy as np
import pandas as pd
from pyomo.environ import *
os.chdir('/Users/yul/Desktop/Github Fetch/stochastic_programming/MnM Project')

In [3]:
# Create the model
model = ConcreteModel()
opt = SolverFactory('gurobi_ampl')
model.dual = Suffix(direction=Suffix.IMPORT)

# Define decision Variables
X = []; Y = []; Z = []
for i in range(20):
    X.append('x{0}'.format(i))
    for j in range(5):
        Y.append('y{0}{1}'.format(i,j))
        Z.append('z{0}{1}'.format(i,j))

model.x = Var(X,domain=NonNegativeIntegers)
model.y = Var(Y,domain=NonNegativeIntegers)
model.z = Var(Z,domain=NonNegativeIntegers)

# Define variables for the amount of nutrient, cooking time and expenses
model.fplus = {(i): Var(name='fplus_{0}'.format(i), domain=NonNegativeReals) for i in range(5)}
model.fminus = {(i): Var(name='fminus_{0}'.format(i), domain=NonNegativeReals) for i in range(5)}
model.eplus = Var(name='eplus', domain=NonNegativeReals)
model.eminus = Var(name='eminus', domain=NonNegativeReals)
model.w = Var(name='w', domain=NonNegativeReals)

In [4]:
nutrition = pd.read_csv('nutrition.csv')
nutrition = nutrition.sort_values(by='title').reset_index()
nutrition.head(5)

,index,Unnamed: 0,title,calories,fat,protein,sodium
0,19,113,Baked Feta and Greens with Lemony Yogurt,577.0,38.0,23.0,839.0
1,12,49,"Butternut Squash, Kale, and Crunchy Pepitas Taco",233.0,17.0,7.0,363.0
2,15,77,"Curried Lentil, Tomato, and Coconut Soup",437.0,28.0,13.0,667.0
3,17,104,Curried Pumpkin Soup,165.0,14.0,2.0,373.0
4,8,29,Easy General Tso's Chicken,699.0,35.0,50.0,1189.0


In [19]:
svd = pd.read_csv('full_afterSVD.csv')
svd = svd.tail(1)
svd = pd.DataFrame(svd.T[2:])
svd = svd.reset_index()
svd = svd.sort_values(by='index').reset_index()
svd.head(5)

,level_0,index,53
0,2,Baked.Feta.and.Greens.with.Lemony.Yogurt,1.000000
1,18,Butternut.Squash..Kale..and.Crunchy.Pepitas.Taco.,1.000000
2,9,Curried.Lentil..Tomato..and.Coconut.Soup.,1.000000
3,10,Curried.Pumpkin.Soup,-0.340423
4,4,Easy.General.Tso.s.Chicken,1.000000


In [6]:
nutrition['rating'] = svd[53]

In [7]:
nutrition.head(5)

,index,Unnamed: 0,title,calories,fat,protein,sodium,rating
0,19,113,Baked Feta and Greens with Lemony Yogurt,577.0,38.0,23.0,839.0,1.000000
1,12,49,"Butternut Squash, Kale, and Crunchy Pepitas Taco",233.0,17.0,7.0,363.0,1.000000
2,15,77,"Curried Lentil, Tomato, and Coconut Soup",437.0,28.0,13.0,667.0,1.000000
3,17,104,Curried Pumpkin Soup,165.0,14.0,2.0,373.0,-0.340423
4,8,29,Easy General Tso's Chicken,699.0,35.0,50.0,1189.0,1.000000


In [8]:
# Define parameters
# recipe matrix 
rating = nutrition['rating'].tolist()
calories = nutrition['calories'].tolist()
protein = nutrition['protein'].tolist()
fat = nutrition['fat'].tolist()
sodium = nutrition['sodium'].tolist()

# We have to determine :
price = [9.38, 11.97 , 7.26 ,4.97 , 11.75 , 7.99 , 5.01,14.87 ,10.28 , 4.38, 9.38 ,  8.14 ,  5.32, 11.37,10.23, 7.64 , 9.45 , 6.76, 4.27 ,12.87 ,3.99]
p_times = [120,30 , 60 ,42, 84, 54,36,150,102, 30,78,90,30,120,90, 60,108,90,30,120 ,60]
available_time_y = [70,50,100,100,120]
available_time_z = [150,120,100,80,80]

# lower(1) and upper(2) bound of nutrient
c_bound=[2700, 3600]
p_bound= [ 140 ,200]
f_bound=[150 , 250]
s_bound= [2400 ,3000]
e_bound= 25

# parameter for objective
alpha = 0.1

In [20]:
# look up objective funtion
print(sum(rating[i]*model.x['x{0}'.format(i)] for i in range(20)))

x[x0] + x[x1] + x[x2] - 0.340423452347904*x[x3] + x[x4] + x[x5] + x[x6] + 0.510428711502824*x[x7] + x[x8] - 0.0996332091329677*x[x9] + 0.639974486744004*x[x10] + 0.42621364666303097*x[x11] - 0.734373515118672*x[x12] + x[x13] - 0.20872622423873802*x[x14] + 1.34014595247377*x[x15] + 2.3392994497990496*x[x16] - 0.0594419260750856*x[x17] + 1.1007243559255901*x[x18] + 0.46643253273179797*x[x19]


In [10]:
# Define Objective
model.opj = Objective(expr = sum(rating[i]*model.x['x{0}'.format(i)] for i in range(20)) 
             - alpha*model.w, sense=maximize)

In [11]:
#Define constraints
# -- Meal Constraints
model.meals = ConstraintList()
model.meals.add(sum(model.x['x{0}'.format(i)] for i in range(20)) == 5)
model.meals.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 5)
model.meals.add(sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 5)
for i in range(20):
    model.meals.add(sum(model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)] for j in range(5)) == model.x['x{0}'.format(i)])
        
# -- Nutrition Bounds
model.nutrition = ConstraintList()
model.nutrition.add(sum(calories[i]*model.x['x{0}'.format(i)] for i in range(20)) >= c_bound[0])
model.nutrition.add(sum(protein[i]*model.x['x{0}'.format(i)] for i in range(20)) >= p_bound[0])
model.nutrition.add(sum(fat[i]*model.x['x{0}'.format(i)] for i in range(20)) >= f_bound[0])
model.nutrition.add(sum(sodium[i]*model.x['x{0}'.format(i)] for i in range(20)) >= s_bound[0])

model.nutrition.add(sum(calories[i]*model.x['x{0}'.format(i)] for i in range(20)) <= c_bound[1])
model.nutrition.add(sum(protein[i]*model.x['x{0}'.format(i)] for i in range(20)) <= p_bound[1])
model.nutrition.add(sum(fat[i]*model.x['x{0}'.format(i)] for i in range(20)) <= f_bound[1])
model.nutrition.add(sum(sodium[i]*model.x['x{0}'.format(i)] for i in range(20)) <= s_bound[1])

# -- Number of recipe
model.number_rec = ConstraintList()
model.number_rec.add(sum(model.x['x{0}'.format(i)] for i in range(20))== 
                       sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) + 
                       sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5))
                      )

# -- Time Inequity
model.p_time = ConstraintList()
model.p_time.add(sum(p_times[i]* (model.y['y{0}{1}'.format(i,j)] - model.z['z{0}{1}'.format(i,j)]) 
                                        for i in range(20) for j in range(5)) <= model.w)
model.p_time.add(sum(p_times[i]* (-model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)]) 
                                        for i in range(20) for j in range(5)) <= model.w)
        
# -- No redundant recipe constraint
model.red_rec = ConstraintList()
for j in range(5):
    model.red_rec.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20)) <= 1)
    model.red_rec.add(sum(model.z['z{0}{1}'.format(i,j)] for i in range(20)) <= 1)
for i in range(20):    
    model.red_rec.add(sum(model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)] for j in range(5)) <= 1)
    
# -- Preparation time constraint
model.prepare_time = ConstraintList()
for j in range(5):
    model.prepare_time.add(sum(p_times[i]*model.y['y{0}{1}'.format(i,j)] for i in range(20)) <= available_time_y[j])
    model.prepare_time.add(sum(p_times[i]*model.z['z{0}{1}'.format(i,j)] for i in range(20)) <= available_time_z[j])        

In [12]:
# Solve the problem
results = opt.solve(model,tee=True)
results.write()

Academic license - for non-commercial use only
Gurobi 8.1.0# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 74
  Number of variables: 221
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Gurobi 8.1.0\x3a optimal solution; objective 2.4449004214243817; 301 simplex iterations; 45 branch-and-cut nodes; plus 1 simplex iteration for intbasis
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.15741896629333496
# --------------------------------------------------

In [23]:
# Store solution in the model.
model.solutions.store_to(results)
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 74
  Number of variables: 221
  Sense: unknown
Solver: 
- Status: ok
  Message: Gurobi 8.1.0\x3a optimal solution; objective 2.4449004214243817; 301 simplex iterations; 45 branch-and-cut nodes; plus 1 simplex iteration for intbasis
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.15741896629333496
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: None
  Status: optimal
  Message: Gurobi 8.1.0\x3a optimal solution; objective 2.4449004214243817; 301 simplex iterations; 45 branch-and-cut nodes; plus 1 simplex iteration for intbasis
  Objective:
    opj:
      Value: 2.4449004214243812
  Variable:
    w:
      Value: 18
    x[x10]:
      Value: 1
    x[x12]:
      Value: 1
    x[x16]:
      Value: 1
    x[x5]:
      Value: 1
    x[x6]:
      Value: 1
    y[y102]:
      Value: 1
    y[y124]:
      Value: 1
    y[y61]:
      Value: 1
    z[z161]:
  